In [1]:
from math import sqrt
import pandas as pd
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import datetime
from sklearn import metrics
from numpy import concatenate
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [2]:
dataset = pd.read_csv('dataset.csv')
dataset.set_index(['date','sex','age','height','timezone'], inplace=True)
dataset.head()
#dataset.shape

,,,,,steps,bedin,bedout,sleep duration,deepduration,lightduration
date,sex,age,height,timezone,,,,,,
2016-04-02,1.0,39.0,174.0,1.0,2547.55,23.78,7.82,7.76,3.24,4.49
2016-04-03,1.0,39.0,174.0,1.0,329.53,23.41,6.48,6.93,2.54,4.38
2016-04-05,1.0,39.0,174.0,1.0,9031.89,22.95,6.73,7.19,3.04,4.14
2016-04-06,1.0,39.0,174.0,1.0,15331.70,22.44,6.18,7.61,3.51,4.14
2016-04-07,1.0,39.0,174.0,1.0,5222.45,24.57,4.04,3.38,1.92,1.49


In [3]:
#Normalizzazione
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

x_data = x_scaler.fit_transform(dataset[['steps','bedin','bedout','sleep duration','deepduration','lightduration']])
y_data = y_scaler.fit_transform(dataset[['sleep duration']])